In [ ]:
from Crypto.Cipher import AES
import os

KEY = None


class StepUpCounter(object):
    def __init__(self, step_up=False):
        self.value = os.urandom(16).hex()
        self.step = 1
        self.stup = step_up

    def increment(self):
        if self.stup:
            self.newIV = hex(int(self.value, 16) + self.step)
        else:
            self.newIV = hex(int(self.value, 16) - self.stup)
        self.value = self.newIV[2:len(self.newIV)]
        return bytes.fromhex(self.value.zfill(32))

    def __repr__(self):
        self.increment()
        return self.value



# @chal.route('/bean_counter/encrypt/')
def encrypt():
    cipher = AES.new(KEY, AES.MODE_ECB)
    ctr = StepUpCounter()

    out = []
    with open("challenge_files/bean_flag.png", 'rb') as f:
        block = f.read(16)
        while block:
            keystream = cipher.encrypt(ctr.increment())
            xored = [a^b for a, b in zip(block, keystream)]
            out.append(bytes(xored).hex())
            block = f.read(16)

    return {"encrypted": ''.join(out)}

In [14]:
from Crypto.Cipher import AES
import os
import requests

ENCODE_URL = "https://aes.cryptohack.org/bean_counter/encrypt/"

def get_respone(url):
    respone = requests.get(url).json()
    keys = respone.keys()
    for k in ("ciphertext", "encrypted"):
        if k in keys:
            return respone[k]
        
def hex_XOR(a, b):
    assert len(a) == len(b)
    return bytes(x ^ y for x, y in zip(bytes.fromhex(a), bytes.fromhex(b))).hex()

def write2png(hex_str, out_filename="flag.png"):
    with open(out_filename, "wb") as f:
        f.write(PNG_HDR16)
        f.write(bytes.fromhex(hex_str))

    # return out_filename

# Bài này cùng counter nên dễ phá.
ciphertext = get_respone(ENCODE_URL)

# Với file PNG, ta biết 16 byte đầu tiên rõ ràng (PNG signature + IHDR):
PNG_HDR16 = b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR'

flag_hex = ""
KEY = hex_XOR(ciphertext[:32], PNG_HDR16.hex())
print(KEY)

for i in range(32, len(ciphertext), 32):
    temp = len(ciphertext[i:i+32])
    flag_hex += hex_XOR(KEY[:temp], ciphertext[i:i+32])

write2png(flag_hex)


09530b0276f63851b1bf76b0939489e8
